# tf.keras.layers

Deep neural network are composed of several hidden layers. tf.keras.layers supplies lots useful API:

    1. Dense, Flatten, Input, DenseFeature, Dropout, etc.
    2. Conv2D, Maxpooling2D, Conv1D, etc.
    3. Embedding, GRU, LSTM, Bidirectional, etc.
    4. Lambda for custom layer, which is only for layer without trainable parameters
    
The layer APIs are friendly to use. 

We talk more about how to define a custom layer in this notebook.
    1. For the layer with trainable variables, we define a class inherit from class Layer
    2. for the layer without trainable variables, we use Lambda to define the layer.

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

## 1. Lambda

If there is no trainable variable in the layer, we do not need to consider the backward propagation process. So using Lambda will be simple and direct.

In [3]:
layer_pow = layers.Lambda(lambda x: tf.math.pow(x, 2))

In [4]:
layer_pow(tf.range(5))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 0,  1,  4,  9, 16], dtype=int32)>

## 2. custom Layer class

To define a custom layer, we have to override the __init__, build, call functions.

In [6]:
class Linear(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
    
    # define the trainable variables in build function
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
        super(Linear, self).build(input_shape)
    
    # define the forward operators in call function
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    # overrider get_config to make the layer compatible with Functional API
    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config

In [7]:
linear = Linear(units = 8)
print(linear.built)


linear.build(input_shape = (None,16)) 
print(linear.built)

False
True


In [8]:
linear = Linear(units = 8)
print(linear.built)
linear.build(input_shape = (None,16)) 
print(linear.compute_output_shape(input_shape = (None,16)))

False
(None, 8)


In [9]:
linear = Linear(units = 16)
print(linear.built)

linear(tf.random.uniform((100,64))) 
print(linear.built)
config = linear.get_config()
print(config)

False
True
{'name': 'linear_2', 'trainable': True, 'dtype': 'float32', 'units': 16}


In [10]:
tf.keras.backend.clear_session()

model = models.Sequential()
model.add(Linear(units = 16,input_shape = (64,)))  
print("model.input_shape: ",model.input_shape)
print("model.output_shape: ",model.output_shape)
model.summary()

model.input_shape:  (None, 64)
model.output_shape:  (None, 16)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear (Linear)              (None, 16)                1040      
Total params: 1,040
Trainable params: 1,040
Non-trainable params: 0
_________________________________________________________________
